# Quiz 3 Hamoye Internship- Machine Learning: Classification - Managing the Quality Metric of Global Ecological Footprint
### Name: Peter Wainaina
### Student ID: 16cc46827641f000    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as pyo
import plotly.express as px

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV

pyo.init_notebook_mode()

df = pd.read_csv("Global-Ecological-Footprint_main_classification.csv")

#### Question 1
According to a use-case, in a certain ML task, a false positive is six times costlier than a false negative. You, as a Data Scientist, trained 4 models, to solve the use case.

Keep the following evaluation criteria in mind:

1) Must have a recall rate of at least 80% 

2) Must have a false positive rate of 8% or less 

3) Must minimize business costs

 After creating each binary classification model, you generated the corresponding confusion matrix. Which confusion matrix represents the model that satisfies the requirements? 

**Answer:** TN = 98%, FP = 2%, FN = 18%, TP = 82%

#### Question 2:
You are working on a spam classification system using regularized logistic regression. “Spam” is a positive class (y = 1) and “not spam” is the negative class (y = 0). You have trained your classifier and there are n = 1700 examples in the test set. The confusion matrix of predicted class vs. actual class is:

What is the F1 score of this classifier?

**Answer:** 0.2635

working in the cell below:

In [2]:
tp = 255 # true positive
fp = 1380 # false positive
tn = 20 # true negative
fn = 45 # false negative

# calculating the precision
precision = (tp)/(tp+fp)

# calculating the recall
recall = (tp)/(tp+fn)

# calculating the F1 score using the precision and recall
F1 = 2 * (precision * recall)/(precision + recall)

print(f"The F1 Score is {F1}")

The F1 Score is 0.2635658914728682


#### Question 3: 
Jack is working on classification modelling. While evaluating the model, he saw that the difference between test and training error is a big positive number with a low training error. Which of the following, is he currently facing?


**Answer:** Overfitting

#### Question 4: 
Which of the following metric is generally NOT useful for a classification problem?

**Answer:** RMSE Value

#### Question 5:
Why do we use weak learners in boosting, instead of strong learners?

**Answer:** To prevent overfitting

#### Question 6:
You are building a classifier and the accuracy is poor on both the training and test sets. Which would you use to try to improve the performance?

**Answer:** Boosting 

#### Question 7:
A classifier predicts if insurance claims are fraudulent or not. The cost of paying a fraudulent claim is higher than the cost of investigating a claim that is suspected to be fraudulent. Which metric should we use to evaluate this classifier?

**Answer:** Precision

#### Question 8:
The ROC curve above was generated from a classification algorithm. What can we say about this classifier?

**Answer:** The model has no discrimination capacity to differentiate between the positive and the negative class

#### Question 9:
A random forest classifier was used to classify handwritten digits 0-9 into the numbers they were intended to represent. The confusion matrix below was generated from the results. Based on the matrix, which number was predicted with the least accuracy?

**Answer:** 8

#### Question 10:
A medical company is building a model to predict the occurrence of thyroid cancer. The training data contains 900 negative instances (people who don't have cancer) and 100 positive instances. The resulting model has 90% accuracy, but extremely poor recall. What steps can be used to improve the model's performance? (SELECT TWO)

**Answer:** 

Generate synthetic samples/data using SMOTE

Collect more data for the positive case

#### Question 11:
You are developing a machine learning classification algorithm that categorizes handwritten digits 0-9 into the numbers they represent. How should you pre-process the label data?

**Answer:** Normalization

#### Question 12:
What is the entropy of the target variable if its actual values are given as:

[1,0,1,1,0,1,0]


**Answer:** - 3/7 log(3/7) - 4/7 log(4/7)

#### Question 13:
Which of these is not a good metric for evaluating classification algorithms for data with imbalanced class problems?

**Answer:** Accuracy

In [3]:
df.shape

(10000, 14)

In [4]:
df['stabf'].value_counts()

stabf
unstable    6380
stable      3620
Name: count, dtype: int64

In [5]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [7]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [8]:
#dropping "stab" as per the instructions
df = df.drop('stab', axis=1)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [9]:
df = pd.get_dummies(df)
df.reset_index(drop=True, inplace=True)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf_stable,stabf_unstable
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,False,True
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,True,False
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,False,True
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,False,True
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,False,True


In [10]:
df = df.drop("stabf_stable", axis=1)
df = df.rename(columns = {"stabf_unstable":"stabf"})
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,True
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,False
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,True
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,True
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,True


In [11]:
X = df.drop(columns= "stabf")
y = df["stabf"]

In [12]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stabf'],
      dtype='object')

In [13]:
X = df.drop('stabf', axis=1)
y = df['stabf']

In [14]:
#split the data into training and testing sets with a random state of 1 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )

In [15]:
y_train.value_counts()

stabf
True     5092
False    2908
Name: count, dtype: int64

In [16]:
df['stabf'] = df['stabf'].replace({'unstable': 1, 'stable': 0})
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,True
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,False
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,True
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,True
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,True


In [17]:
df.corr()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
tau1,1.000000,0.015586,-0.005970,-0.017265,0.027183,-0.015485,-0.015924,-0.015807,0.010521,0.015350,-0.001279,0.005494,0.234898
tau2,0.015586,1.000000,0.014273,-0.001965,-0.004769,0.006573,0.007673,-0.005963,-0.001742,0.015383,0.016508,-0.011764,0.246280
tau3,-0.005970,0.014273,1.000000,0.004354,0.016953,-0.003134,-0.008780,-0.017531,-0.011605,0.007671,0.014702,-0.011497,0.237492
tau4,-0.017265,-0.001965,0.004354,1.000000,-0.003173,0.010553,0.006169,-0.011211,-0.004149,0.008431,0.003260,-0.000491,0.239375
p1,0.027183,-0.004769,0.016953,-0.003173,1.000000,-0.573157,-0.584554,-0.579239,0.000721,0.015405,0.001069,-0.015451,0.009938
p2,-0.015485,0.006573,-0.003134,0.010553,-0.573157,1.000000,0.002388,-0.006844,0.015603,-0.018032,0.007555,0.019817,0.006173
p3,-0.015924,0.007673,-0.008780,0.006169,-0.584554,0.002388,1.000000,0.012953,-0.003219,-0.011575,-0.005897,-0.010485,-0.000649
p4,-0.015807,-0.005963,-0.017531,-0.011211,-0.579239,-0.006844,0.012953,1.000000,-0.013636,0.002850,-0.003515,0.017505,-0.022785
g1,0.010521,-0.001742,-0.011605,-0.004149,0.000721,0.015603,-0.003219,-0.013636,1.000000,0.007559,-0.005836,0.012431,0.197664
g2,0.015350,0.015383,0.007671,0.008431,0.015405,-0.018032,-0.011575,0.002850,0.007559,1.000000,-0.012809,-0.014909,0.217341


In [18]:
df['stabf'].value_counts()

stabf
True     6380
False    3620
Name: count, dtype: int64

In [19]:
# Scaling the training and testing data for the independent variables
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(X_train)
Xtrain_scaled

array([[ 0.36732671, -0.98604156,  0.65044706, ...,  0.33985949,
         0.58556788,  0.49223946],
       [-0.06465869,  0.08943734,  1.03507899, ..., -1.5584875 ,
         1.42964862, -1.44352101],
       [-1.46785   ,  1.29841758, -0.50253617, ...,  1.45153362,
        -1.04574277,  0.49248925],
       ...,
       [ 0.65760851, -0.72275633, -1.4058879 , ...,  0.29310048,
        -1.55058661,  0.81034412],
       [-0.05931596, -1.26053241, -1.01047147, ..., -0.38825455,
        -0.72678059,  1.66791568],
       [-1.47321368,  0.63843757,  0.25012249, ..., -1.17410957,
         1.179282  ,  0.78362657]])

In [20]:
Xtest_scaled = scaler.fit_transform(X_test)
Xtest_scaled

array([[ 0.59216281, -0.42056489,  1.47247233, ...,  1.17528683,
        -1.49264423,  1.08629139],
       [ 0.19918339,  0.36454311, -0.19007599, ..., -0.39594869,
         1.41270302,  1.2275355 ],
       [-1.08603505, -0.32183365, -0.87350524, ..., -1.44437004,
         0.65421559, -1.67979856],
       ...,
       [ 0.94713772, -1.66831304, -1.62907667, ...,  0.12889933,
         0.57628587,  1.32084757],
       [-1.12735376,  0.18457336, -0.23820702, ...,  0.80017191,
        -1.3484627 , -0.79985771],
       [-1.38555998,  1.49904306,  0.27297057, ..., -0.92058109,
         0.00903865,  1.19055888]])

#### Question 14:
What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

**Answer:** 0.9255

The working is in the cell below

In [21]:
#random forest classifier
rand_clf = RandomForestClassifier(random_state=0)

# model fitting
rand_clf.fit(Xtrain_scaled, y_train)

#using the test data to predict the target variable 
y_pred_rand = rand_clf.predict(Xtest_scaled)

#evaluating the model accuracy:
rf_accuracy = accuracy_score(y_test, y_pred_rand)
print(f"Accuracy of the random forest classifier is {rf_accuracy}")

Accuracy of the random forest classifier is 0.9255


#### Question 15
What is the accuracy on the test set using the XGboost classifier? In 4 decimal places.

**Answer:** 0.9195

The working is in the cell below:

In [22]:
# XGBoostClassifier
Xg = XGBClassifier(n_estimators=200, max_depth=2, learning_rate=0.1, random_state=1)

# model fitting
Xg.fit(Xtrain_scaled, y_train)

#using the test data to predict the target variable 
Xg_pred = Xg.predict(Xtest_scaled)

#evaluating the model accuracy:
xg_accuracy = accuracy_score(y_test, Xg_pred)
print(f"Accuracy of the XGBoost classifier is {xg_accuracy}")

Accuracy of the XGBoost classifier is 0.9195


#### Question 16
What is the accuracy on the test set using the LGBM classifier? In 4 decimal places

**Answer:** 0.9375

The working is in the cell below:

In [23]:
# LGBMClassifier
lgbm = lgb.LGBMClassifier(learning_rate=0.1, max_depth=7, random_state=1)

# model fitting
lgbm.fit(Xtrain_scaled, y_train)

#using the test data to predict the target variable 
lgbm_pred = lgbm.predict(Xtest_scaled)

#Evaluating the model accuracy:
lg_accuracy = accuracy_score(y_test, lgbm_pred)
lg_accuracy
print(f"Accuracy of the LGBM classifier is {lg_accuracy}")

Accuracy of the LGBM classifier is 0.9375


#### Question 17:
Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

**Answer:** N_estimators = 1000 , min_samples_split = 2 , min_samples_leaf = 8, max_features = None

The working is in the cells below:

In [24]:
# ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=1)

# model fitting
etc.fit(Xtrain_scaled, y_train)

# using the model to predict the test data
etc_pred = etc.predict(Xtest_scaled)

etc_accuracy = accuracy_score(y_test, etc_pred)
etc_accuracy
print(f"The accuracy of the ExtraTreesClassifier is {etc_accuracy}")

The accuracy of the ExtraTreesClassifier is 0.926


In [25]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

In [26]:
rs = RandomizedSearchCV(estimator=etc, param_distributions=hyperparameter_grid, 
                        cv=5, n_iter=10, scoring="accuracy", n_jobs=-1, 
                        verbose=1, random_state=1)
search = rs.fit(Xtrain_scaled, y_train) 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/pierre/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/pierre/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/pierre/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the 

In [27]:
#best parameters
search.best_params_.values()

dict_values([1000, 2, 8, None])

#### Question 18:
Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?

**Answer:** Lower

The working is in the cells below:

In [28]:
etc_2 = ExtraTreesClassifier(n_estimators= 100, min_samples_split=2, 
                             min_samples_leaf=8, max_features=None, 
                             random_state=1)
etc_2.fit(Xtrain_scaled, y_train)
etc2_pred = etc_2.predict(Xtest_scaled)

In [29]:
#evaluating the accuracy of the new model

etc_accuracy2 = accuracy_score(y_test, etc2_pred)
etc_accuracy2
print(f"The accuracy of the other ExtraTreesClassifier is {etc_accuracy2}")

The accuracy of the other ExtraTreesClassifier is 0.9245


#### Question 19
What other hyperparameter optimization method can you try apart from Random Search?

**Answer:** Grid Search

#### Question 20
Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least important respectively?

**Answer:** tau2, p1

The working is in the cells below:

In [30]:
#the optimum model (ExtraTreesClassifier)
etc

ExtraTreesClassifier(random_state=1)

In [31]:
etc.feature_importances_

array([0.11739736, 0.11844468, 0.11316851, 0.11546569, 0.03950675,
       0.04037132, 0.04070628, 0.04057864, 0.08978291, 0.09367636,
       0.09688268, 0.09401882])

In [32]:
feature_dict = dict(zip(df.columns, list(etc.feature_importances_)))
feature_dict

{'tau1': 0.11739736493320078,
 'tau2': 0.11844468079199041,
 'tau3': 0.11316851246674982,
 'tau4': 0.11546569217199552,
 'p1': 0.039506754233827476,
 'p2': 0.04037131556379323,
 'p3': 0.040706278296424536,
 'p4': 0.040578638540044426,
 'g1': 0.08978290601483987,
 'g2': 0.09367635844551439,
 'g3': 0.09688268324346265,
 'g4': 0.09401881529815702}

In [33]:
#Creating a pandas dataframe from the dictionary feature_dict
feature_df = pd.DataFrame(feature_dict, index=[0])

#Transposing the dataframe to visualize the feature importance
feature_df = feature_df.T

feature_df

,0
tau1,0.117397
tau2,0.118445
tau3,0.113169
tau4,0.115466
p1,0.039507
p2,0.040371
p3,0.040706
p4,0.040579
g1,0.089783
g2,0.093676
